In [1]:
# @title Default title text
import yfinance as yf
import pandas as pd
from datetime import datetime
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import talib
!pip install ta
import ta
#!pip install --upgrade yfinance
#!pip install mplfinance
import os

# # Example: delete 'myfile.csv'
# print(os.listdir('/content'))
# !rm -rf /content/*
# print("All saved files deleted successfully!")

In [2]:
# @title Swing Trading-Day
#Swing Trading

import yfinance as yf
import pandas as pd
import numpy as np
from ta.trend import ADXIndicator
from ta.volatility import BollingerBands
from ta.volatility import AverageTrueRange

# ✅ Symbol and Interval
nifty = "^NSEI"     # Example stock   --^NSEI
#nifty_S =  ["TITAN.NS" , "HCLTECH.NS","HAL.NS","PFC.NS"]    # Example stock
interval = "1d"        # Use "1m" for intraday
period="60d"   #"2d"  #2mo
# ✅ Fetch data
df = yf.download(nifty, period=period, interval=interval)
df.columns = df.columns.get_level_values(0)

# ✅ Timezone & Resampling
if interval == "1m":
    df.index = df.index.tz_convert("Asia/Kolkata")
    df["Date"] = df.index.normalize()  # keep datetime64[ns]
    df_3min = df.resample("3min").agg({
        "Open": "first",
        "High": "max",
        "Low": "min",
        "Close": "last",
        "Volume": "sum"
    }).dropna()
    df_3min["Date"] = df_3min.index.normalize()
else:
    df.index = pd.to_datetime(df.index)
    df["Date"] = df.index.normalize()
    df_3min = df.copy()

# ✅ Remove ambiguity by resetting index if needed
if 'Date' in df_3min.index.names:
    df_3min.reset_index(inplace=True, drop=True)

# ✅ Rounding and Indicators
df_3min["EMA_5"] = round(df_3min["Close"].ewm(span=5).mean())
df_3min["EMA_7"] = round(df_3min["Close"].ewm(span=7).mean())
df_3min["EMA_10"] = round(df_3min["Close"].ewm(span=10).mean())
df_3min["EMA_20"] = round(df_3min["Close"].ewm(span=20).mean())
df_3min["EMA_50"] = round(df_3min["Close"].ewm(span=50).mean())
df_3min["EMA_100"] = round(df_3min["Close"].ewm(span=100).mean())
df_3min["EMA_200"] = round(df_3min["Close"].ewm(span=200).mean())
df_3min["Pivot"] = (df_3min["High"] + df_3min["Low"] + df_3min["Close"]) / 3
df_3min["Close"] = round(df_3min["Close"])
df_3min["Open"] = round(df_3min["Open"])
df_3min["High"] = round(df_3min["High"])
df_3min["Low"] = round(df_3min["Low"])
df_3min["Volume"] = round(df_3min["Volume"])
df_3min["Avg_volume"] = round(df_3min["Volume"].rolling(window=10).mean())

def classify_volume(row):
    if row["Volume"] > 1.5 * row["Avg_volume"]:
        return "HIGH_H"
    elif row["Volume"] > 2 * row["Avg_volume"]:
        return "HIGH_HH"
    elif row["Volume"] > row["Avg_volume"]:
        return "High"
    else:
        return "Normal"

df_3min["Vol_incr"] = df_3min.apply(classify_volume, axis=1)
# ✅ Variance Calculation
df_3min["Variance"] = round(df_3min["Close"].rolling(window=10).var())
variance_threshold = round(df_3min["Variance"].mean() * 1.5, 2)
df_3min["Var_Th"] = variance_threshold

# ✅ Rolling Highs and Lows
df_3min["Max_High_5"] = df_3min["High"].rolling(window=5).max()
df_3min["Min_Low_5"] = df_3min["Low"].rolling(window=5).min()

# ✅ Previous Day High, Low, Volume
prev_highs = df_3min.groupby("Date")["High"].max().shift(1)
prev_lows = df_3min.groupby("Date")["Low"].min().shift(1)
prev_avg_vol = df_3min.groupby("Date")["Volume"].mean().shift(1)

# ✅ Map previous values to each row
df_3min["Prev_High"] = df_3min["Date"].map(prev_highs)
df_3min["Prev_Low"] = df_3min["Date"].map(prev_lows)
df_3min["Prev_Avg_Vol"] = df_3min["Date"].map(prev_avg_vol)


# ✅ ATR Calculation
def atr(df, window=10):
    high_low = df["High"] - df["Low"]
    high_close = np.abs(df["High"] - df["Close"].shift(1))
    low_close = np.abs(df["Low"] - df["Close"].shift(1))
    tr = np.maximum(high_low, np.maximum(high_close, low_close))
    return tr.rolling(window).mean()



# ✅ Supertrend Calculation
def supertrend(df, period=10, multiplier=3):
    df["ATR"] = atr(df, period)
    hl2 = (df["High"] + df["Low"]) / 2
    df["Upper_Band"] = hl2 + (multiplier * df["ATR"])
    df["Lower_Band"] = hl2 - (multiplier * df["ATR"])
    df["Supertrend"] = np.nan
    uptrend = True


    for i in range(1, len(df)):
        if df["Close"].iloc[i] > df["Upper_Band"].iloc[i-1]:
            uptrend = True
        elif df["Close"].iloc[i] < df["Lower_Band"].iloc[i-1]:
            uptrend = False

        df.loc[df.index[i], "Supertrend"] = df["Lower_Band"].iloc[i] if uptrend else df["Upper_Band"].iloc[i]

    return df

df_3min = supertrend(df_3min)
df_3min["ATR"] = atr(df_3min, 10)



lookback=14
# ✅ Compute RSI, MACD
df_3min["RSI"] = round(ta.momentum.rsi(df_3min["Close"], window=14))
df_3min["MACD"] = ta.trend.macd(df_3min["Close"])
df_3min["MACD_Signal"] = ta.trend.macd_signal(df_3min["Close"])

df_3min["HH"] = df_3min["High"].rolling(window=lookback).max()
df_3min["LL"] = df_3min["Low"].rolling(window=lookback).min()

# ✅ Compute Bollinger Bands
bb = BollingerBands(close=df_3min["Close"], window=20, window_dev=2)
df_3min["Upper_BB"] = bb.bollinger_hband()
df_3min["Lower_BB"] = bb.bollinger_lband()

df_3min['volatility'] = ta.volatility.bollinger_hband_indicator(df_3min['Close'], window=20)

# ✅ Compute ADX and Directional Indicators
adx = ADXIndicator(high=df_3min["High"], low=df_3min["Low"], close=df_3min["Close"], window=14)
df_3min["ADX"] = round(adx.adx())
df_3min["+DI"] = round( adx.adx_pos(),2)
df_3min["-DI"] =round(adx.adx_neg(),2)

# ✅ Detect ADX Increasing for the Last 5 Candles
df_3min["ADX_Incr"] = df_3min["ADX"].diff(5) > 0
df_3min["ADX_Decreasing"] = df_3min["ADX"].diff(5) < 0
df_3min['DI_Diff'] = abs(df_3min['+DI'] - df_3min['-DI'])  # Compute absolute difference

# ✅ Compute Pivot Points
df_3min["Pivot"] = round((df_3min["High"] + df_3min["Low"] + df_3min["Close"]) / 3,0)
df_3min["R1"] = round((2 * df_3min["Pivot"]) - df_3min["Low"], 0)
df_3min["S1"] = round((2 * df_3min["Pivot"]) - df_3min["High"], 0)
df_3min["R2"] = round(df_3min["Pivot"] + (df_3min["High"] - df_3min["Low"]), 0)
df_3min["S2"] = round(df_3min["Pivot"] - (df_3min["High"] - df_3min["Low"]), 0)
df_3min["Golden_Cross"] = (df_3min["EMA_20"] > df_3min["EMA_50"]) & (df_3min["EMA_20"].shift(1) < df_3min["EMA_50"].shift(1))
df_3min["Death_Cross"] = (df_3min["EMA_20"] < df_3min["EMA_50"]) & (df_3min["EMA_20"].shift(1) > df_3min["EMA_50"].shift(1))
df_3min["Golden/Death"] = np.where(df_3min["EMA_20"] > df_3min["EMA_50"],
                             np.where(df_3min["EMA_20"].shift(1) < df_3min["EMA_50"].shift(1), "BUY", "No Signal"),
                             np.where(df_3min["EMA_20"].shift(1) > df_3min["EMA_50"].shift(1), "SELL", "No Signal"))
df_3min["PriceAction-Buy"]=(df_3min["Close"] > df_3min["Open"]) & (df_3min["Open"].shift(1) > df_3min["Close"].shift(1)) & (df_3min["Close"] > df_3min["Open"].shift(1)) & (df_3min["Open"] < df_3min["Close"].shift(1))



df_3min["PriceAction_Sell"] = (
        (df_3min["Close"] < df_3min["Open"]) &
        (df_3min["Open"].shift(1) < df_3min["Close"].shift(1)) &
        (df_3min["Close"] < df_3min["Open"].shift(1)) &
        (df_3min["Open"] > df_3min["Close"].shift(1))
    )
df_3min["Doji"] = (
        abs(df_3min["Close"] - df_3min["Open"]) / df_3min["Open"] < 0.1
    )

df_3min["Price"] = "WAIT"

def detect_swing_highs_lows3(df, lookback=5):
    df['Swing_High'] = np.nan
    df['Swing_Low'] = np.nan

    for i in range(lookback, len(df) - lookback):
        if df['High'].iloc[i] > df['High'].iloc[i - lookback:i].max() and df['High'].iloc[i] > df['High'].iloc[i + 1:i + 1 + lookback].max():
            df.at[df.index[i], 'Swing_High'] = round(df['High'].iloc[i], 0)

        if df['Low'].iloc[i] < df['Low'].iloc[i - lookback:i].min() and df['Low'].iloc[i] < df['Low'].iloc[i + 1:i + 1 + lookback].min():
            df.at[df.index[i], 'Swing_Low'] = round(df['Low'].iloc[i], 0)

    return df






# Step 1: Detect Swing Highs and Lows
def detect_swing_highs_lows(df, lookback=2):
    df = df.copy()
    df['Swing_High'] = np.nan
    df['Swing_Low'] = np.nan

    for i in range(lookback, len(df) - lookback):
        window_highs = df['High'].iloc[i - lookback:i + lookback + 1]
        window_lows = df['Low'].iloc[i - lookback:i + lookback + 1]

        center = i

        if df['High'].iloc[center] == window_highs.max():
            df.at[df.index[center], 'Swing_High'] = round(df['High'].iloc[center], 2)

        if df['Low'].iloc[center] == window_lows.min():
            df.at[df.index[center], 'Swing_Low'] = round(df['Low'].iloc[center], 2)

    return df


# Step 2: Extract Support/Resistance Levels
def extract_sr_levels_from_swings(df, lookback=2, tolerance=0.5):
    swing_highs = df['Swing_High'].dropna().tail(lookback)
    swing_lows = df['Swing_Low'].dropna().tail(lookback)

    resistance_levels = []
    support_levels = []


    for price in swing_highs:
        if not any(abs(price - existing) <= tolerance for existing in resistance_levels):
            resistance_levels.append(price)

    for price in swing_lows:
        if not any(abs(price - existing) <= tolerance for existing in support_levels):
            support_levels.append(price)

    resistance_levels.sort(reverse=True)
    support_levels.sort()

    return resistance_levels, support_levels


# Step 3: Apply to your DataFrame (df_3min assumed to be your 3-minute interval data)
df_3min = detect_swing_highs_lows(df_3min, lookback=2)
res_levels, sup_levels = extract_sr_levels_from_swings(df_3min, lookback=2, tolerance=0.5)

# Step 4: Assign the most recent levels to the latest row
df_3min.loc[df_3min.index[-1], "res_level"] = res_levels[0] if res_levels else np.nan
df_3min.loc[df_3min.index[-1], "sup_level"] = sup_levels[0] if sup_levels else np.nan

df_3min["S-R"] = df_3min.apply(
    lambda row: f"{round(row['res_level'], 0)} - {round(row['sup_level'], 0)}"
    if pd.notna(row['res_level']) and pd.notna(row['sup_level']) else np.nan,
    axis=1
)

# # Set "BUY" signal where the PriceAction-Buy condition is True
df_3min.loc[df_3min["PriceAction-Buy"] == True, "Signal"] = "BUY"

# # Set "SELL" signal where the PriceAction-Sell condition is True, but only if "BUY" is not already set
# df_3min.loc[(df_3min["PriceAction_Sell"] == True) & (df_3min["Signal"] != "BUY"), "Signal"] = "SELL"

# # Set "WAIT" signal where Doji condition is True, but only if no other signal is present
df_3min.loc[(df_3min["Doji"] == True) & (df_3min["Signal"] == "WAIT"), "Signal"] = "WAIT"

def compute_heikin_ashi(df):
    ha_df = df.copy()
    ha_df['HA_Close'] = (df['Open'] + df['High'] + df['Low'] + df['Close']) / 4
    ha_df['HA_Open'] = (df['Open'].shift(1) + df['Close'].shift(1)) / 2

   # ha_df['HA_Open'].fillna((df['Open'] + df['Close']) / 2, inplace=True)
    ha_df['HA_Open'] = ha_df['HA_Open'].fillna((df['Open'] + df['Close']) / 2)
    ha_df['HA_High'] = ha_df[['High', 'HA_Open', 'HA_Close']].max(axis=1)
    ha_df['HA_Low'] = ha_df[['Low', 'HA_Open', 'HA_Close']].min(axis=1)
    return ha_df
def detect_engulfing_and_wicks(df):
    conditions = []

    for i in range(1, len(df)):
        row_prev = df.iloc[i - 1]
        row_curr = df.iloc[i]

        # Bullish Engulfing
        is_bullish_engulf = (
            row_prev['HA_Close'] < row_prev['HA_Open'] and
            row_curr['HA_Close'] > row_curr['HA_Open'] and
            row_curr['HA_Close'] > row_prev['HA_Open'] and
            row_curr['HA_Open'] < row_prev['HA_Close']
        )

        # Bearish Engulfing
        is_bearish_engulf = (
            row_prev['HA_Close'] > row_prev['HA_Open'] and
            row_curr['HA_Close'] < row_curr['HA_Open'] and
            row_curr['HA_Open'] > row_prev['HA_Close'] and
            row_curr['HA_Close'] < row_prev['HA_Open']
        )

        # Rejection wicks (Pin bar style)
        body = abs(row_curr['HA_Close'] - row_curr['HA_Open'])
        upper_wick = row_curr['HA_High'] - max(row_curr['HA_Close'], row_curr['HA_Open'])
        lower_wick = min(row_curr['HA_Close'], row_curr['HA_Open']) - row_curr['HA_Low']

        # Swing High pattern (rejection on top)
        swing_high = upper_wick > 2 * body and row_curr['HA_Close'] < row_curr['HA_Open']

        # Swing Low pattern (rejection at bottom)
        swing_low = lower_wick > 2 * body and row_curr['HA_Close'] > row_curr['HA_Open']

        conditions.append({
            'BullishEngulfing': is_bullish_engulf,
            'BearishEngulfing': is_bearish_engulf,
            'SwingHighWick': swing_high,
            'SwingLowWick': swing_low
        })

    return pd.DataFrame(conditions, index=df.index[1:])

def get_potential_swings(df):
    ha_df = compute_heikin_ashi(df)
    patterns = detect_engulfing_and_wicks(ha_df)

    ha_df = ha_df.iloc[1:]  # because patterns start from index 1
    ha_df = ha_df.join(patterns)

    ha_df['HA_SW_High'] = ha_df['BearishEngulfing'] | ha_df['SwingHighWick']
    ha_df['HA_SW_Low'] = ha_df['BullishEngulfing'] | ha_df['SwingLowWick']
    return ha_df
df_3min = get_potential_swings(df_3min)

def detect_heikin_ashi_trend(df, lookback=3):
    trends = []

    for i in range(len(df)):
        if i < lookback:
            trends.append(None)
            continue

        recent = df.iloc[i-lookback+1:i+1]

        # Count bullish/bearish candles
        bullish = (recent['HA_Close'] > recent['HA_Open']).sum()
        bearish = (recent['HA_Close'] < recent['HA_Open']).sum()

        if bullish == lookback:
            trends.append("Uptrend")
        elif bearish == lookback:
            trends.append("Downtrend")
        else:
            trends.append("Sideways")

    df["HA_Trend"] = trends
    return df
df_3min = detect_heikin_ashi_trend(df_3min)
df_3min["HA_Close"] = (df_3min["Open"] + df_3min["High"] + df_3min["Low"] + df_3min["Close"]) / 4
df_3min["HA_Open"] = df_3min["HA_Close"].shift(1)

df_3min["SMHT"] = (df_3min["MACD"] - df_3min["MACD"].rolling(window=10).mean()) / df_3min["MACD"].rolling(window=10).std()

# ✅ Buy/Sell Ranges
df_3min["Buy_Range"] = df_3min["S1"]  # Buying near S1
df_3min["Sell_Range"] = df_3min["R1"]  # Selling near R1

# Candle parameters
df_3min["Candle_Body"] = abs(df_3min["Close"] - df_3min["Open"])
df_3min["Prev_Candle_Body"] = df_3min["Candle_Body"].shift(1)
df_3min["Is_Bullish"] = df_3min["Close"] > df_3min["Open"]
df_3min["Is_Bearish"] = df_3min["Close"] < df_3min["Open"]


def generate_signals_with_dynamic_ranges(data):
    signals, scores, entries, targets, stops, rr_ratios = [], [], [], [], [], []

    for i in range(len(data)):
        row = data.iloc[i]
        close = row["Close"]
        open_ = row['Open']
        adx = row["ADX"]
        prev_row = data.iloc[i - 1]
        plus_di, minus_di = row["+DI"], row["-DI"]
        candle_body = row["Candle_Body"]
        prev_body = row["Prev_Candle_Body"]
        prev_close = prev_row["Close"]
        prev_high = prev_row["High"]
        prev_low = prev_row["Low"]
        variance = row["Variance"]
        supertrend = row["Supertrend"]
        ema5, ema10, ema20, ema50, ema200 = row["EMA_5"], row["EMA_10"], row["EMA_20"], row["EMA_50"], row["EMA_200"]
        upper_bb, lower_bb = row["Upper_BB"], row["Lower_BB"]
        adx_incr = row["ADX_Incr"]
        macd, macd_signal = row["MACD"], row["MACD_Signal"]
        smht = row["SMHT"]
        ha_close, ha_open = row["HA_Close"], row["HA_Open"]
        r1 = row.get("R1", None)
        s1 = row.get("S1", None)
        atr = row["ATR"]

        signal = "No Signal"
        entry = target = stop = None
        score = 0
               # --- Score Logic ---
        if adx > 25: score += 1 + (0.5 if adx_incr else 0)
        if plus_di > minus_di: score += 1
        if close > supertrend: score += 1.5
        if ema5 > ema10 > ema20: score += 1.5
        if macd > macd_signal: score += 1
        if variance > variance_threshold and close > upper_bb: score += 1.5
        if ema50 > ema200: score += 1
        if smht > 1 and ha_close > ha_open: score += 1.5
        if minus_di > plus_di: score -= 1
        if close < supertrend: score -= 1.5
        if ema5 < ema10 < ema20: score -= 1.5
        if macd < macd_signal: score -= 1
        if variance > variance_threshold and close < lower_bb: score -= 1.5
        if ema50 < ema200: score -= 1
        if smht < -1 and ha_close < ha_open: score -= 1.5

        # --- Signal Classification ---
        risk_buffer = close * 0.0075  # ~0.75% buffer
        vola_boost = variance * 0.1




        if score >= 4 :
            signal = f"Strong BUY ({round(score, 1)})"
            entry = close + close * 0.001  # buffer
            stop = lower_bb if lower_bb else (s1 if s1 else close - risk_buffer - vola_boost)
            target = upper_bb if upper_bb else (r1 if r1 else close + (close - stop) * 2)

        elif 2 <= score < 4:
            signal = f"BUY ({round(score, 1)})"
            entry = close + close * 0.0005
            stop = lower_bb if lower_bb else close - risk_buffer
            target = upper_bb if upper_bb else close + (close - stop) * 1.5

        elif score <= -4:
            signal = f"Strong SELL ({round(score, 1)})"
            entry = close # close - close * 0.001
            stop = upper_bb if upper_bb else (r1 if r1 else close + risk_buffer + vola_boost)
            if stop <= entry:
                risk = close * 0.01
                stop = entry + risk
            else:
                risk = stop - entry
            raw_target = entry - risk * 2
            target = lower_bb if lower_bb and lower_bb < entry else (
                s1 if s1 and s1 < entry else raw_target
            )

        elif -4 < score <= -2:
            signal = f"SELL ({round(score, 1)})"
            entry = close - close * 0.0005
            stop = upper_bb if upper_bb else (r1 if r1 else close + risk_buffer + vola_boost)
            if stop <= entry:
                risk = close * 0.01
                stop = entry + risk
            else:
                risk = stop - entry
            raw_target = entry - risk * 2
            target = lower_bb if lower_bb and lower_bb < entry else (
                s1 if s1 and s1 < entry else raw_target
            )



        # Reward/Risk
        rr = round((target - entry) / (entry - stop), 2) if entry and stop and target else ""



        signals.append(signal)
        scores.append(score)
        entries.append(round(entry, 2) if entry else "")
        targets.append(round(target, 2) if target else "")
        stops.append(round(stop, 2) if stop else "")
        rr_ratios.append(rr)

    return signals, scores, entries, targets, stops, rr_ratios

current_close = df_3min['Close'].iloc[-1]
current_high = df_3min['High'].iloc[-1]
current_low = df_3min['Low'].iloc[-1]

# 🔹 Get recent valid swing points (last N)
recent_swing_highs = df_3min['Swing_High'].dropna().tail(5)
recent_swing_lows = df_3min['Swing_Low'].dropna().tail(5)

def generate_smht_signal(df):
    signals = []
    for smht in df["SMHT"]:
        if smht > 2:
            signals.append("Strong Buy")
        elif smht < -2:
            signals.append("Strong Sell")
        else:
            signals.append("")
    return signals

# Usage:
df_3min["SMHT_Signal"] = generate_smht_signal(df_3min)

df_3min["HA_SW_High"] = df_3min["HA_SW_High"].apply(lambda x: "T" if x else "F")
df_3min["HA_SW_Low"]= df_3min["HA_SW_Low"].apply(lambda x: "T" if x else "F")
df_3min["Signal"], df_3min["Score"], df_3min["Entry"], df_3min["Target"], df_3min["StopLoss"], df_3min["RR"] = generate_signals_with_dynamic_ranges(df_3min)
#df_3min["EMA_20>50"] = np.where(df_3min["EMA_20"] > df_3min["EMA_50"], "T", "F")
df_3min["EMA_Status"] = (
    np.where(df_3min["EMA_5"] > df_3min["EMA_10"], "T", "F") +
    np.where(df_3min["EMA_10"] > df_3min["EMA_20"], "T", "F") +
    np.where(df_3min["EMA_20"] > df_3min["EMA_50"], "T", "F") +
    np.where(df_3min["EMA_50"] > df_3min["EMA_100"], "T", "F")
)
Entry_condition = (
    (df_3min["EMA_Status"] == "TTTT") &
    (df_3min["HA_Trend"] == "Uptrend") &
    --(df_3min["MACD"] > df_3min["MACD_Signal"]) &
    (df_3min["ADX"] > 20) &
    (df_3min["+DI"] > df_3min["-DI"])
)

Exit_condition = (
    (df_3min["EMA_Status"] == "FFFF") &
    (df_3min["HA_Trend"] == "Downtrend") &
    (df_3min["MACD"] < df_3min["MACD_Signal"]) &
    (df_3min["ADX"] > 20) &
    (df_3min["+DI"] < df_3min["-DI"])
)
df_3min["EE_Signal"] = np.where(Entry_condition, "Buy",
                       np.where(Exit_condition, "Sell", ""))


df_3min["Yest_H/L"]=""
#Based on the Swing high and low giving the signals
# ✅ Add yesterday's high and low columns
df_3min['Prev_High'] = df_3min['High'].shift(1)
df_3min['Prev_Low'] = df_3min['Low'].shift(1)
df_3min['Prev_Close'] = df_3min['Close'].shift(1)

df_3min['Prev_High_Fill'] = df_3min['Swing_High'].ffill()
df_3min['Prev_Low_Fill'] = df_3min['Swing_Low'].ffill()

# Breakout conditions
df_3min['Break_High'] = (df_3min['Close'] > df_3min['Prev_High_Fill']) & (df_3min['Close'] > df_3min['Prev_Low_Fill'])
df_3min['Break_Low'] = (df_3min['Close'] < df_3min['Prev_Low_Fill']) & (df_3min['Close'] < df_3min['Prev_High_Fill'])
df_3min['Break_Prevs_SwingLow'] = (df_3min['Close'] > df_3min['Prev_Low_Fill']) & (df_3min['Close'] < df_3min['Prev_High_Fill'])
df_3min['Body'] = abs(df_3min['Close'] - df_3min['Open'])
df_3min['Strong_Break_High'] = df_3min['Break_High'] & (df_3min['Body'] > df_3min['ATR'] * 0.8)
atr_multiplier = 1.5
df_3min['ATR_Break_High'] = df_3min['Close'] > (df_3min['Prev_High_Fill'] + atr_multiplier * df_3min['ATR'])
df_3min['ATR_Break_Low'] = df_3min['Close'] < (df_3min['Prev_Low_Fill'] - atr_multiplier * df_3min['ATR'])
df_3min['Breakout_with_Momentum'] = (df_3min['Break_High']) & (df_3min['RSI'] > 60)
df_3min['Inside_Bar'] = (df_3min['High'] < df_3min['Prev_High']) & (df_3min['Low'] > df_3min['Prev_Low'])

conditions =[
    df_3min['Break_High'],
    df_3min['Break_Low'],
    df_3min['Break_Prevs_SwingLow'],
    df_3min['Strong_Break_High'],
    df_3min['ATR_Break_High'],
    df_3min['ATR_Break_Low'],
    df_3min['Breakout_with_Momentum']
    ]


choices = ['SwingHIGH-Breakout-BUY', 'SWingLow-Breakdown-SELL','Swinglow_Breakout-ShortBuy','Strong_Break_High-BUY','ATR_Break_High-BUY','ATR_Break_Low-SELL','Breakout_with_Momentum-StrongBUY']

df_3min['Swing_Signal'] = np.select(conditions, choices, default='Hold')



# ✅ Check for breakouts based on yesterday's high and low
df_3min['Yesterday_High_Breaks_Close'] = df_3min['Close'] >df_3min['Prev_High']
df_3min['Yesterday_Low_Breaks_Close'] = df_3min['Close'] < df_3min['Prev_Low']
df_3min['Yesterday_Low_Breaks_Low'] = df_3min['Low'] < df_3min['Prev_Low']
df_3min['Yesterday_High_Beaks_PrevHigh'] = df_3min['High'] > df_3min['Prev_High']
df_3min['Yesterday_Low_Beaks_PrevHigh'] = df_3min['Low'] > df_3min['Prev_High']
df_3min['Today_High_Breaks_SwingHigh'] = df_3min['Close'] > df_3min['Prev_High_Fill']
df_3min['Today_High_Breaks_SwingLow'] = df_3min['Low'] < df_3min['Prev_Low_Fill']
df_3min['Today_CloseBreaks_BB_Upper'] = df_3min['Close'] > df_3min['Upper_BB']
df_3min['Today_LowBreakdown_BB_Lower'] = df_3min['Low'] < df_3min['Lower_BB']

df_3min['Yest_H/L'] = np.where(df_3min['Yesterday_High_Breaks_Close'], 'Current Close breaks Previous High- BUY',
                             np.where(df_3min['Yesterday_Low_Breaks_Close'], 'Current Close lessthan Previous Low -Sell',
                              np.where(df_3min['Today_High_Breaks_SwingHigh'], 'Current Close breaks Swingh High -BUY',
                               np.where(df_3min['Today_High_Breaks_SwingLow'], 'Current Close breakdown Swing low -Sell',
                                  np.where(df_3min['Yesterday_High_Beaks_PrevHigh'], 'Current High Breakout PrevHigh -BUY',
                                         np.where(df_3min['Yesterday_Low_Beaks_PrevHigh'], 'Current Low Breakout PrevHigh -BUY',
                                                  np.where(df_3min['Today_CloseBreaks_BB_Upper'], 'Current Close Breakout BB_UPPER -BUY',
                                                             np.where(df_3min['Today_LowBreakdown_BB_Lower'], 'Current Low Breakdown BB_Lower -SELL',
                             np.where(df_3min['Yesterday_Low_Breaks_Low'], 'Current Low lessthan Previous Low -Sell', 'No Break')))))))))


df_3min["strong_buy"] = (
    (df_3min["EMA_5"] > df_3min["EMA_10"]) &
    (df_3min["EMA_10"] > df_3min["EMA_20"]) &
    (df_3min["ADX"] > 25) &
    (df_3min["+DI"] > df_3min["-DI"]) &
    (df_3min["Supertrend"] < df_3min["Close"]) &
    #(df_3min["RSI"] < 60) &
    (df_3min["HA_SW_Low"] == "T") &
    (df_3min["Close"] > df_3min["S1"])  # Confirmed above support

    )
df_3min["strong_sell"] = (
    (df_3min["EMA_5"] < df_3min["EMA_10"]) &
    (df_3min["EMA_10"] < df_3min["EMA_20"]) &
    (df_3min["ADX"] > 25) &
    (df_3min["-DI"] > df_3min["+DI"]) &
    (df_3min["Supertrend"] > df_3min["Close"]) &
    #(df_3min["RSI"] > 40) &
    (df_3min["HA_SW_High"] == "T") &
    (df_3min["Close"] < df_3min["R1"])  # Rejected at resistance
)
df_3min["Final_Signal"] = np.where(
    df_3min["strong_buy"], "STRONG BUY",
    np.where(df_3min["strong_sell"], "STRONG SELL", df_3min["Yest_H/L"])
    )
# Use previous row's S1 and R1
df_3min["S1_prev"] = df_3min["S1"].shift(1)
df_3min["R1_prev"] = df_3min["R1"].shift(1)


# Step 1: Calculate Swing High/Low (exclude current candle)
df_3min["Swing_High_5"] = df_3min["High"].shift(1).rolling(window=5).max()
df_3min["Swing_Low_5"] = df_3min["Low"].shift(1).rolling(window=5).min()



#df_today=df_3min[["Close","High","Low", "+DI", "-DI","DI_Diff","Variance", "S1", "R1", "S2","R2","HA_Signal", "ADX", "ADX_Incr", "Signal", "EMA_5", "EMA_10", "EMA_20","Golden/Death"]]
#df_3min=df_3min[["Open","Close","High","Low", "Signal", "Score","S1","R1","S2","R2","Signal_S/R","Entry","Target","StopLoss","ADX_Incr","ADX", "+DI", "-DI", "Variance", "EMA_5", "EMA_10"]]
df_3min=df_3min[["Date","Close","High","Low", "Signal","S1","R1","Swing_Signal","Vol_incr","Final_Signal","Swing_High","Swing_Low","ADX_Incr","ADX", "RSI","+DI", "-DI","HA_SW_High","HA_SW_Low","HA_Trend"]]
# filename = f"nifty50BB_{nifty}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
# df_3min.to_csv(filename)
#df_3min = df_3min[(df_3min["EE_Signal"] == "Buy") & (df_3min["HA_SW_Low"] == "T")] # for buy
#df_3min = df_3min[(df_3min["EE_Signal"] == "Buy")] # for buy
#df_3min = df_3min[(df_3min["HA_SW_Low"] == "T")] # for buy
#df_3min = df_3min[(df_3min["HA_SW_High"] == "T")] # for sell
df_3min.tail(10)



YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


,Date,Close,High,Low,Signal,S1,R1,Swing_Signal,Vol_incr,Final_Signal,Swing_High,Swing_Low,ADX_Incr,ADX,RSI,+DI,-DI,HA_SW_High,HA_SW_Low,HA_Trend
50,2025-04-22,24167.0,24243.0,24072.0,Strong BUY (4.5),24079.0,24250.0,SwingHIGH-Breakout-BUY,Normal,Current Close breaks Swingh High -BUY,NaN,NaN,False,23.0,66.0,44.00,23.42,F,F,Uptrend
51,2025-04-23,24329.0,24359.0,24120.0,Strong BUY (4.5),24179.0,24418.0,SwingHIGH-Breakout-BUY,Normal,Current Close breaks Previous High- BUY,NaN,NaN,True,24.0,68.0,44.23,22.25,F,F,Uptrend
52,2025-04-24,24247.0,24348.0,24216.0,BUY (3.0),24192.0,24324.0,SwingHIGH-Breakout-BUY,Normal,Current Close breaks Swingh High -BUY,NaN,NaN,True,25.0,66.0,42.95,21.60,F,F,Sideways
53,2025-04-25,24039.0,24365.0,23848.0,BUY (3.0),23803.0,24320.0,SwingHIGH-Breakout-BUY,Normal,Current Close lessthan Previous Low -Sell,NaN,23848.0,True,24.0,62.0,38.26,27.01,F,F,Sideways
54,2025-04-28,24328.0,24355.0,24054.0,Strong BUY (4.5),24137.0,24438.0,SwingHIGH-Breakout-BUY,Normal,Current Close breaks Swingh High -BUY,NaN,NaN,True,24.0,65.0,35.70,25.20,F,T,Sideways
55,2025-04-29,24336.0,24458.0,24291.0,Strong BUY (4.5),24266.0,24433.0,SwingHIGH-Breakout-BUY,Normal,Current Close breaks Swingh High -BUY,NaN,NaN,False,23.0,65.0,36.65,24.27,F,F,Sideways
56,2025-04-30,24334.0,24396.0,24199.0,BUY (3.0),24224.0,24421.0,SwingHIGH-Breakout-BUY,High,Current Close breaks Swingh High -BUY,NaN,NaN,False,23.0,65.0,35.02,25.27,F,F,Sideways
57,2025-05-02,24347.0,24589.0,24238.0,BUY (3.0),24193.0,24544.0,Swinglow_Breakout-ShortBuy,High,Current High Breakout PrevHigh -BUY,24589.0,NaN,False,23.0,65.0,36.59,23.28,F,T,Sideways
58,2025-05-05,24461.0,24526.0,24401.0,BUY (3.0),24400.0,24525.0,Swinglow_Breakout-ShortBuy,Normal,No Break,NaN,NaN,False,23.0,67.0,35.07,22.32,F,F,Sideways
59,2025-05-06,24380.0,24510.0,24332.0,BUY (3.0),24304.0,24482.0,Swinglow_Breakout-ShortBuy,Normal,Current Close lessthan Previous Low -Sell,NaN,NaN,False,23.0,65.0,33.58,23.02,T,F,Sideways


In [ ]:
# @title Day Trading-Options or Stocks


#Day Trading or Options trading
import yfinance as yf
import pandas as pd
import numpy as np
from ta.trend import ADXIndicator
from ta.volatility import BollingerBands
from ta.volatility import AverageTrueRange

# ✅ Symbol and Interval
nifty = "^NSEI"    # "^NSEI"       # Example stock
nifty_S =  ["TITAN.NS" , "HCLTECH.NS","HAL.NS","PFC.NS"]    # Example stock
interval = "1m"        # Use "1m" for intraday
period="5d"   #"2d"  #2mo
# ✅ Fetch data
df = yf.download(nifty, period=period, interval=interval)
df.columns = df.columns.get_level_values(0)

# ✅ Timezone & Resampling
if interval == "1m":
    df.index = df.index.tz_convert("Asia/Kolkata")
    df["Date"] = df.index.normalize()  # keep datetime64[ns]
    df_3min = df.resample("3min").agg({
        "Open": "first",
        "High": "max",
        "Low": "min",
        "Close": "last",
        "Volume": "sum"
    }).dropna()
    df_3min["Date"] = df_3min.index.normalize()
else:
    df.index = pd.to_datetime(df.index)
    df["Date"] = df.index.normalize()
    df_3min = df.copy()

# ✅ Remove ambiguity by resetting index if needed
if 'Date' in df_3min.index.names:
    df_3min.reset_index(inplace=True, drop=True)

# ✅ Rounding and Indicators
df_3min["EMA_5"] = round(df_3min["Close"].ewm(span=5).mean())
df_3min["EMA_7"] = round(df_3min["Close"].ewm(span=7).mean())
df_3min["EMA_10"] = round(df_3min["Close"].ewm(span=10).mean())
df_3min["EMA_20"] = round(df_3min["Close"].ewm(span=20).mean())
df_3min["EMA_50"] = round(df_3min["Close"].ewm(span=50).mean())
df_3min["EMA_100"] = round(df_3min["Close"].ewm(span=100).mean())
df_3min["EMA_200"] = round(df_3min["Close"].ewm(span=200).mean())
df_3min["Pivot"] = (df_3min["High"] + df_3min["Low"] + df_3min["Close"]) / 3
df_3min["Close"] = round(df_3min["Close"])
df_3min["Open"] = round(df_3min["Open"])
df_3min["High"] = round(df_3min["High"])
df_3min["Low"] = round(df_3min["Low"])
df_3min["Volume"] = round(df_3min["Volume"])
df_3min["Avg_volume"] = round(df_3min["Volume"].rolling(window=10).mean())

def classify_volume(row):
    if row["Volume"] > 1.5 * row["Avg_volume"]:
        return "HIGH_H"
    elif row["Volume"] > 2 * row["Avg_volume"]:
        return "HIGH_HH"
    elif row["Volume"] > row["Avg_volume"]:
        return "High"
    else:
        return "Normal"

df_3min["Vol_incr"] = df_3min.apply(classify_volume, axis=1)
# ✅ Variance Calculation
df_3min["Variance"] = round(df_3min["Close"].rolling(window=10).var())
variance_threshold = round(df_3min["Variance"].mean() * 1.5, 2)
df_3min["Var_Th"] = variance_threshold

# ✅ Rolling Highs and Lows
df_3min["Max_High_5"] = df_3min["High"].rolling(window=5).max()
df_3min["Min_Low_5"] = df_3min["Low"].rolling(window=5).min()

# ✅ Previous Day High, Low, Volume
prev_highs = df_3min.groupby("Date")["High"].max().shift(1)
prev_lows = df_3min.groupby("Date")["Low"].min().shift(1)
prev_avg_vol = df_3min.groupby("Date")["Volume"].mean().shift(1)

# ✅ Map previous values to each row
df_3min["Prev_High"] = df_3min["Date"].map(prev_highs)
df_3min["Prev_Low"] = df_3min["Date"].map(prev_lows)
df_3min["Prev_Avg_Vol"] = df_3min["Date"].map(prev_avg_vol)


# ✅ ATR Calculation
def atr(df, period=10):
    high_low = df["High"] - df["Low"]
    high_close = np.abs(df["High"] - df["Close"].shift(1))
    low_close = np.abs(df["Low"] - df["Close"].shift(1))
    tr = np.maximum(high_low, np.maximum(high_close, low_close))
    return tr.rolling(period).mean()

# ✅ Supertrend Calculation
def supertrend(df, period=10, multiplier=3):
    df["ATR"] = atr(df, period)
    hl2 = (df["High"] + df["Low"]) / 2
    df["Upper_Band"] = hl2 + (multiplier * df["ATR"])
    df["Lower_Band"] = hl2 - (multiplier * df["ATR"])
    df["Supertrend"] = np.nan
    uptrend = True


    for i in range(1, len(df)):
        if df["Close"].iloc[i] > df["Upper_Band"].iloc[i-1]:
            uptrend = True
        elif df["Close"].iloc[i] < df["Lower_Band"].iloc[i-1]:
            uptrend = False

        df.loc[df.index[i], "Supertrend"] = df["Lower_Band"].iloc[i] if uptrend else df["Upper_Band"].iloc[i]

    return df

df_3min = supertrend(df_3min)



lookback=14
# ✅ Compute RSI, MACD
df_3min["RSI"] = round(ta.momentum.rsi(df_3min["Close"], window=14))
df_3min["MACD"] = ta.trend.macd(df_3min["Close"])
df_3min["MACD_Signal"] = ta.trend.macd_signal(df_3min["Close"])

df_3min["HH"] = df_3min["High"].rolling(window=lookback).max()
df_3min["LL"] = df_3min["Low"].rolling(window=lookback).min()

# ✅ Compute Bollinger Bands
bb = BollingerBands(close=df_3min["Close"], window=20, window_dev=2)
df_3min["Upper_BB"] = bb.bollinger_hband()
df_3min["Lower_BB"] = bb.bollinger_lband()

df_3min['volatility'] = ta.volatility.bollinger_hband_indicator(df_3min['Close'], window=20)

# ✅ Compute ADX and Directional Indicators
adx = ADXIndicator(high=df_3min["High"], low=df_3min["Low"], close=df_3min["Close"], window=14)
df_3min["ADX"] = round(adx.adx())
df_3min["+DI"] = round( adx.adx_pos(),2)
df_3min["-DI"] =round(adx.adx_neg(),2)

# ✅ Detect ADX Increasing for the Last 5 Candles
df_3min["ADX_Incr"] = df_3min["ADX"].diff(5) > 0
df_3min["ADX_Decreasing"] = df_3min["ADX"].diff(5) < 0
df_3min['DI_Diff'] = abs(df_3min['+DI'] - df_3min['-DI'])  # Compute absolute difference

# ✅ Compute Pivot Points
df_3min["Pivot"] = round((df_3min["High"] + df_3min["Low"] + df_3min["Close"]) / 3,0)
df_3min["R1"] = round((2 * df_3min["Pivot"]) - df_3min["Low"], 0)
df_3min["S1"] = round((2 * df_3min["Pivot"]) - df_3min["High"], 0)
df_3min["R2"] = round(df_3min["Pivot"] + (df_3min["High"] - df_3min["Low"]), 0)
df_3min["S2"] = round(df_3min["Pivot"] - (df_3min["High"] - df_3min["Low"]), 0)
df_3min["Golden_Cross"] = (df_3min["EMA_20"] > df_3min["EMA_50"]) & (df_3min["EMA_20"].shift(1) < df_3min["EMA_50"].shift(1))
df_3min["Death_Cross"] = (df_3min["EMA_20"] < df_3min["EMA_50"]) & (df_3min["EMA_20"].shift(1) > df_3min["EMA_50"].shift(1))
df_3min["Golden/Death"] = np.where(df_3min["EMA_20"] > df_3min["EMA_50"],
                             np.where(df_3min["EMA_20"].shift(1) < df_3min["EMA_50"].shift(1), "BUY", "No Signal"),
                             np.where(df_3min["EMA_20"].shift(1) > df_3min["EMA_50"].shift(1), "SELL", "No Signal"))
df_3min["PriceAction-Buy"]=(df_3min["Close"] > df_3min["Open"]) & (df_3min["Open"].shift(1) > df_3min["Close"].shift(1)) & (df_3min["Close"] > df_3min["Open"].shift(1)) & (df_3min["Open"] < df_3min["Close"].shift(1))



df_3min["PriceAction_Sell"] = (
        (df_3min["Close"] < df_3min["Open"]) &
        (df_3min["Open"].shift(1) < df_3min["Close"].shift(1)) &
        (df_3min["Close"] < df_3min["Open"].shift(1)) &
        (df_3min["Open"] > df_3min["Close"].shift(1))
    )
df_3min["Doji"] = (
        abs(df_3min["Close"] - df_3min["Open"]) / df_3min["Open"] < 0.1
    )

df_3min["Price"] = "WAIT"

def detect_swing_highs_lows3(df, lookback=5):
    df['Swing_High'] = np.nan
    df['Swing_Low'] = np.nan

    for i in range(lookback, len(df) - lookback):
        if df['High'].iloc[i] > df['High'].iloc[i - lookback:i].max() and df['High'].iloc[i] > df['High'].iloc[i + 1:i + 1 + lookback].max():
            df.at[df.index[i], 'Swing_High'] = round(df['High'].iloc[i], 0)

        if df['Low'].iloc[i] < df['Low'].iloc[i - lookback:i].min() and df['Low'].iloc[i] < df['Low'].iloc[i + 1:i + 1 + lookback].min():
            df.at[df.index[i], 'Swing_Low'] = round(df['Low'].iloc[i], 0)

    return df






# Step 1: Detect Swing Highs and Lows
def detect_swing_highs_lows(df, lookback=2):
    df = df.copy()
    df['Swing_High'] = np.nan
    df['Swing_Low'] = np.nan

    for i in range(lookback, len(df) - lookback):
        window_highs = df['High'].iloc[i - lookback:i + lookback + 1]
        window_lows = df['Low'].iloc[i - lookback:i + lookback + 1]

        center = i

        if df['High'].iloc[center] == window_highs.max():
            df.at[df.index[center], 'Swing_High'] = round(df['High'].iloc[center], 2)

        if df['Low'].iloc[center] == window_lows.min():
            df.at[df.index[center], 'Swing_Low'] = round(df['Low'].iloc[center], 2)

    return df


# Step 2: Extract Support/Resistance Levels
def extract_sr_levels_from_swings(df, lookback=2, tolerance=0.5):
    swing_highs = df['Swing_High'].dropna().tail(lookback)
    swing_lows = df['Swing_Low'].dropna().tail(lookback)

    resistance_levels = []
    support_levels = []


    for price in swing_highs:
        if not any(abs(price - existing) <= tolerance for existing in resistance_levels):
            resistance_levels.append(price)

    for price in swing_lows:
        if not any(abs(price - existing) <= tolerance for existing in support_levels):
            support_levels.append(price)

    resistance_levels.sort(reverse=True)
    support_levels.sort()

    return resistance_levels, support_levels


# Step 3: Apply to your DataFrame (df_3min assumed to be your 3-minute interval data)
df_3min = detect_swing_highs_lows(df_3min, lookback=2)
res_levels, sup_levels = extract_sr_levels_from_swings(df_3min, lookback=2, tolerance=0.5)

# Step 4: Assign the most recent levels to the latest row
df_3min.loc[df_3min.index[-1], "res_level"] = res_levels[0] if res_levels else np.nan
df_3min.loc[df_3min.index[-1], "sup_level"] = sup_levels[0] if sup_levels else np.nan

df_3min["S-R"] = df_3min.apply(
    lambda row: f"{round(row['res_level'], 0)} - {round(row['sup_level'], 0)}"
    if pd.notna(row['res_level']) and pd.notna(row['sup_level']) else np.nan,
    axis=1
)

# # Set "BUY" signal where the PriceAction-Buy condition is True
df_3min.loc[df_3min["PriceAction-Buy"] == True, "Signal"] = "BUY"

# # Set "SELL" signal where the PriceAction-Sell condition is True, but only if "BUY" is not already set
# df_3min.loc[(df_3min["PriceAction_Sell"] == True) & (df_3min["Signal"] != "BUY"), "Signal"] = "SELL"

# # Set "WAIT" signal where Doji condition is True, but only if no other signal is present
df_3min.loc[(df_3min["Doji"] == True) & (df_3min["Signal"] == "WAIT"), "Signal"] = "WAIT"

def compute_heikin_ashi(df):
    ha_df = df.copy()
    ha_df['HA_Close'] = (df['Open'] + df['High'] + df['Low'] + df['Close']) / 4
    ha_df['HA_Open'] = (df['Open'].shift(1) + df['Close'].shift(1)) / 2

   # ha_df['HA_Open'].fillna((df['Open'] + df['Close']) / 2, inplace=True)
    ha_df['HA_Open'] = ha_df['HA_Open'].fillna((df['Open'] + df['Close']) / 2)
    ha_df['HA_High'] = ha_df[['High', 'HA_Open', 'HA_Close']].max(axis=1)
    ha_df['HA_Low'] = ha_df[['Low', 'HA_Open', 'HA_Close']].min(axis=1)
    return ha_df
def detect_engulfing_and_wicks(df):
    conditions = []

    for i in range(1, len(df)):
        row_prev = df.iloc[i - 1]
        row_curr = df.iloc[i]

        # Bullish Engulfing
        is_bullish_engulf = (
            row_prev['HA_Close'] < row_prev['HA_Open'] and
            row_curr['HA_Close'] > row_curr['HA_Open'] and
            row_curr['HA_Close'] > row_prev['HA_Open'] and
            row_curr['HA_Open'] < row_prev['HA_Close']
        )

        # Bearish Engulfing
        is_bearish_engulf = (
            row_prev['HA_Close'] > row_prev['HA_Open'] and
            row_curr['HA_Close'] < row_curr['HA_Open'] and
            row_curr['HA_Open'] > row_prev['HA_Close'] and
            row_curr['HA_Close'] < row_prev['HA_Open']
        )

        # Rejection wicks (Pin bar style)
        body = abs(row_curr['HA_Close'] - row_curr['HA_Open'])
        upper_wick = row_curr['HA_High'] - max(row_curr['HA_Close'], row_curr['HA_Open'])
        lower_wick = min(row_curr['HA_Close'], row_curr['HA_Open']) - row_curr['HA_Low']

        # Swing High pattern (rejection on top)
        swing_high = upper_wick > 2 * body and row_curr['HA_Close'] < row_curr['HA_Open']

        # Swing Low pattern (rejection at bottom)
        swing_low = lower_wick > 2 * body and row_curr['HA_Close'] > row_curr['HA_Open']

        conditions.append({
            'BullishEngulfing': is_bullish_engulf,
            'BearishEngulfing': is_bearish_engulf,
            'SwingHighWick': swing_high,
            'SwingLowWick': swing_low
        })

    return pd.DataFrame(conditions, index=df.index[1:])

def get_potential_swings(df):
    ha_df = compute_heikin_ashi(df)
    patterns = detect_engulfing_and_wicks(ha_df)

    ha_df = ha_df.iloc[1:]  # because patterns start from index 1
    ha_df = ha_df.join(patterns)

    ha_df['HA_SW_High'] = ha_df['BearishEngulfing'] | ha_df['SwingHighWick']
    ha_df['HA_SW_Low'] = ha_df['BullishEngulfing'] | ha_df['SwingLowWick']
    return ha_df
df_3min = get_potential_swings(df_3min)

def detect_heikin_ashi_trend(df, lookback=3):
    trends = []

    for i in range(len(df)):
        if i < lookback:
            trends.append(None)
            continue

        recent = df.iloc[i-lookback+1:i+1]

        # Count bullish/bearish candles
        bullish = (recent['HA_Close'] > recent['HA_Open']).sum()
        bearish = (recent['HA_Close'] < recent['HA_Open']).sum()

        if bullish == lookback:
            trends.append("Uptrend")
        elif bearish == lookback:
            trends.append("Downtrend")
        else:
            trends.append("Sideways")

    df["HA_Trend"] = trends
    return df
df_3min = detect_heikin_ashi_trend(df_3min)
df_3min["HA_Close"] = (df_3min["Open"] + df_3min["High"] + df_3min["Low"] + df_3min["Close"]) / 4
df_3min["HA_Open"] = df_3min["HA_Close"].shift(1)

df_3min["SMHT"] = (df_3min["MACD"] - df_3min["MACD"].rolling(window=10).mean()) / df_3min["MACD"].rolling(window=10).std()

# ✅ Buy/Sell Ranges
df_3min["Buy_Range"] = df_3min["S1"]  # Buying near S1
df_3min["Sell_Range"] = df_3min["R1"]  # Selling near R1

# Candle parameters
df_3min["Candle_Body"] = abs(df_3min["Close"] - df_3min["Open"])
df_3min["Prev_Candle_Body"] = df_3min["Candle_Body"].shift(1)
df_3min["Is_Bullish"] = df_3min["Close"] > df_3min["Open"]
df_3min["Is_Bearish"] = df_3min["Close"] < df_3min["Open"]


def generate_signals_with_dynamic_ranges(data):
    signals, scores, entries, targets, stops, rr_ratios = [], [], [], [], [], []

    for i in range(len(data)):
        row = data.iloc[i]
        close = row["Close"]
        open_ = row['Open']
        adx = row["ADX"]
        prev_row = data.iloc[i - 1]
        plus_di, minus_di = row["+DI"], row["-DI"]
        candle_body = row["Candle_Body"]
        prev_body = row["Prev_Candle_Body"]
        prev_close = prev_row["Close"]
        prev_high = prev_row["High"]
        prev_low = prev_row["Low"]
        variance = row["Variance"]
        supertrend = row["Supertrend"]
        ema5, ema10, ema20, ema50, ema200 = row["EMA_5"], row["EMA_10"], row["EMA_20"], row["EMA_50"], row["EMA_200"]
        upper_bb, lower_bb = row["Upper_BB"], row["Lower_BB"]
        adx_incr = row["ADX_Incr"]
        macd, macd_signal = row["MACD"], row["MACD_Signal"]
        smht = row["SMHT"]
        ha_close, ha_open = row["HA_Close"], row["HA_Open"]
        r1 = row.get("R1", None)
        s1 = row.get("S1", None)
        atr = row["ATR"]

        signal = "No Signal"
        entry = target = stop = None
        score = 0
               # --- Score Logic ---
        if adx > 25: score += 1 + (0.5 if adx_incr else 0)
        if plus_di > minus_di: score += 1
        if close > supertrend: score += 1.5
        if ema5 > ema10 > ema20: score += 1.5
        if macd > macd_signal: score += 1
        if variance > variance_threshold and close > upper_bb: score += 1.5
        if ema50 > ema200: score += 1
        if smht > 1 and ha_close > ha_open: score += 1.5
        if minus_di > plus_di: score -= 1
        if close < supertrend: score -= 1.5
        if ema5 < ema10 < ema20: score -= 1.5
        if macd < macd_signal: score -= 1
        if variance > variance_threshold and close < lower_bb: score -= 1.5
        if ema50 < ema200: score -= 1
        if smht < -1 and ha_close < ha_open: score -= 1.5

        # --- Signal Classification ---
        risk_buffer = close * 0.0075  # ~0.75% buffer
        vola_boost = variance * 0.1




        if score >= 4 :
            signal = f"Strong BUY ({round(score, 1)})"
            entry = close + close * 0.001  # buffer
            stop = lower_bb if lower_bb else (s1 if s1 else close - risk_buffer - vola_boost)
            target = upper_bb if upper_bb else (r1 if r1 else close + (close - stop) * 2)

        elif 2 <= score < 4:
            signal = f"BUY ({round(score, 1)})"
            entry = close + close * 0.0005
            stop = lower_bb if lower_bb else close - risk_buffer
            target = upper_bb if upper_bb else close + (close - stop) * 1.5

        elif score <= -4:
            signal = f"Strong SELL ({round(score, 1)})"
            entry = close # close - close * 0.001
            stop = upper_bb if upper_bb else (r1 if r1 else close + risk_buffer + vola_boost)
            if stop <= entry:
                risk = close * 0.01
                stop = entry + risk
            else:
                risk = stop - entry
            raw_target = entry - risk * 2
            target = lower_bb if lower_bb and lower_bb < entry else (
                s1 if s1 and s1 < entry else raw_target
            )

        elif -4 < score <= -2:
            signal = f"SELL ({round(score, 1)})"
            entry = close - close * 0.0005
            stop = upper_bb if upper_bb else (r1 if r1 else close + risk_buffer + vola_boost)
            if stop <= entry:
                risk = close * 0.01
                stop = entry + risk
            else:
                risk = stop - entry
            raw_target = entry - risk * 2
            target = lower_bb if lower_bb and lower_bb < entry else (
                s1 if s1 and s1 < entry else raw_target
            )



        # Reward/Risk
        rr = round((target - entry) / (entry - stop), 2) if entry and stop and target else ""



        signals.append(signal)
        scores.append(score)
        entries.append(round(entry, 2) if entry else "")
        targets.append(round(target, 2) if target else "")
        stops.append(round(stop, 2) if stop else "")
        rr_ratios.append(rr)

    return signals, scores, entries, targets, stops, rr_ratios

current_close = df_3min['Close'].iloc[-1]
current_high = df_3min['High'].iloc[-1]
current_low = df_3min['Low'].iloc[-1]

# 🔹 Get recent valid swing points (last N)
recent_swing_highs = df_3min['Swing_High'].dropna().tail(5)
recent_swing_lows = df_3min['Swing_Low'].dropna().tail(5)

def generate_smht_signal(df):
    signals = []
    for smht in df["SMHT"]:
        if smht > 2:
            signals.append("Strong Buy")
        elif smht < -2:
            signals.append("Strong Sell")
        else:
            signals.append("")
    return signals

# Usage:
df_3min["SMHT_Signal"] = generate_smht_signal(df_3min)

df_3min["HA_SW_High"] = df_3min["HA_SW_High"].apply(lambda x: "T" if x else "F")
df_3min["HA_SW_Low"]= df_3min["HA_SW_Low"].apply(lambda x: "T" if x else "F")
df_3min["Signal"], df_3min["Score"], df_3min["Entry"], df_3min["Target"], df_3min["StopLoss"], df_3min["RR"] = generate_signals_with_dynamic_ranges(df_3min)
#df_3min["EMA_20>50"] = np.where(df_3min["EMA_20"] > df_3min["EMA_50"], "T", "F")
df_3min["EMA_Status"] = (
    np.where(df_3min["EMA_5"] > df_3min["EMA_10"], "T", "F") +
    np.where(df_3min["EMA_10"] > df_3min["EMA_20"], "T", "F") +
    np.where(df_3min["EMA_20"] > df_3min["EMA_50"], "T", "F") +
    np.where(df_3min["EMA_50"] > df_3min["EMA_100"], "T", "F")
)
Entry_condition = (
    (df_3min["EMA_Status"] == "TTTT") &
    (df_3min["HA_Trend"] == "Uptrend") &
    --(df_3min["MACD"] > df_3min["MACD_Signal"]) &
    (df_3min["ADX"] > 20) &
    (df_3min["+DI"] > df_3min["-DI"])
)

Exit_condition = (
    (df_3min["EMA_Status"] == "FFFF") &
    (df_3min["HA_Trend"] == "Downtrend") &
    (df_3min["MACD"] < df_3min["MACD_Signal"]) &
    (df_3min["ADX"] > 20) &
    (df_3min["+DI"] < df_3min["-DI"])
)
df_3min["EE_Signal"] = np.where(Entry_condition, "Buy",
                       np.where(Exit_condition, "Sell", ""))


df_3min["Yest_H/L"]=""
# ✅ Add yesterday's high and low columns
df_3min['Prev_High'] = df_3min['High'].shift(1)
df_3min['Prev_Low'] = df_3min['Low'].shift(1)
df_3min['Prev_Close'] = df_3min['Close'].shift(1)

df_3min['Prev_High_Fill'] = df_3min['Swing_High'].ffill()
df_3min['Prev_Low_Fill'] = df_3min['Swing_Low'].ffill()

# df_3min['Break_High'] = (df_3min['Close'] > df_3min['Prev_High_Fill'])
# df_3min['Break_Low'] = (df_3min['Close'] < df_3min['Prev_Low_Fill'])

# Breakout conditions
df_3min['Break_High'] = (df_3min['Close'] > df_3min['Prev_High_Fill']) & (df_3min['Close'] > df_3min['Prev_Low_Fill'])
df_3min['Break_Low'] = (df_3min['Close'] < df_3min['Prev_Low_Fill']) & (df_3min['Close'] < df_3min['Prev_High_Fill'])
df_3min['Break_Prevs_SwingLow'] = (df_3min['Close'] > df_3min['Prev_Low_Fill']) & (df_3min['Close'] < df_3min['Prev_High_Fill'])
df_3min['Body'] = abs(df_3min['Close'] - df_3min['Open'])
df_3min['Strong_Break_High'] = df_3min['Break_High'] & (df_3min['Body'] > df_3min['ATR'] * 0.8)
atr_multiplier = 1.5
df_3min['ATR_Break_High'] = df_3min['Close'] > (df_3min['Prev_High_Fill'] + atr_multiplier * df_3min['ATR'])
df_3min['ATR_Break_Low'] = df_3min['Close'] < (df_3min['Prev_Low_Fill'] - atr_multiplier * df_3min['ATR'])
df_3min['Breakout_with_Momentum'] = (df_3min['Break_High']) & (df_3min['RSI'] > 60)
df_3min['Inside_Bar'] = (df_3min['High'] < df_3min['Prev_High']) & (df_3min['Low'] > df_3min['Prev_Low'])

conditions =[
    df_3min['Break_High'],
    df_3min['Break_Low'],
    df_3min['Break_Prevs_SwingLow'],
    df_3min['Strong_Break_High'],
    df_3min['ATR_Break_High'],
    df_3min['ATR_Break_Low'],
    df_3min['Breakout_with_Momentum']
    ]


choices = ['SwingHigh-Breakout-BUY', 'SWingLow-Breakdown-SELL','Swinglow_Breakout-ShortBuy','Strong_Break_High-BUY','ATR_Break_High-BUY','ATR_Break_Low-SELL','Breakout_with_Momentum-StrongBUY']

df_3min['Swing_Signal'] = np.select(conditions, choices, default='Hold')

# ✅ Check for breakouts based on yesterday's high and low
df_3min['Yesterday_High_Breaks_Close'] = df_3min['Close'] >df_3min['Prev_High']
df_3min['Yesterday_Low_Breaks_Close'] = df_3min['Close'] < df_3min['Prev_Low']
df_3min['Yesterday_Low_Breaks_Low'] = df_3min['Low'] < df_3min['Prev_Low']
df_3min['Yesterday_High_Beaks_PrevHigh'] = df_3min['High'] > df_3min['Prev_High']
df_3min['Yesterday_Low_Beaks_PrevHigh'] = df_3min['Low'] > df_3min['Prev_High']
df_3min['Today_High_Breaks_SwingHigh'] = df_3min['Close'] > df_3min['Prev_High_Fill']
df_3min['Today_High_Breaks_SwingLow'] = df_3min['Low'] < df_3min['Prev_Low_Fill']
df_3min['Today_SwinghLow_Breakout'] = df_3min['Close'] > df_3min['Prev_High_Fill']
df_3min['Today_CloseBreaks_BB_Upper'] = df_3min['Close'] > df_3min['Upper_BB']
df_3min['Today_LowBreakdown_BB_Lower'] = df_3min['Low'] < df_3min['Lower_BB']

df_3min['Yest_H/L'] = np.where(df_3min['Yesterday_High_Breaks_Close'], 'Current Close breaks Previous High- BUY',
                             np.where(df_3min['Yesterday_Low_Breaks_Close'], 'Current Close lessthan Previous Low -Sell',
                              np.where(df_3min['Today_High_Breaks_SwingHigh'], 'Current Close breaks Swingh High -BUY',
                               np.where(df_3min['Today_High_Breaks_SwingLow'], 'Current Close breakdown Swing low -Sell',
                                  np.where(df_3min['Yesterday_High_Beaks_PrevHigh'], 'Current High Breakout PrevHigh -BUY',
                                       np.where(df_3min['Today_SwinghLow_Breakout'], 'Current Close Breaks SwinghLow -BUY',

                                                  np.where(df_3min['Today_CloseBreaks_BB_Upper'], 'Current Close Breakout BB_UPPER -BUY',
                                                             np.where(df_3min['Today_LowBreakdown_BB_Lower'], 'Current Low Breakdown BB_Lower -SELL',
                             np.where(df_3min['Yesterday_Low_Breaks_Low'], 'Current Low lessthan Previous Low -Sell', 'No Break')))))))))


df_3min["strong_buy"] = (
    (df_3min["EMA_5"] > df_3min["EMA_10"]) &
    (df_3min["EMA_10"] > df_3min["EMA_20"]) &
    (df_3min["ADX"] > 25) &
    (df_3min["+DI"] > df_3min["-DI"]) &
    (df_3min["Supertrend"] < df_3min["Close"]) &
    #(df_3min["RSI"] < 60) &
    (df_3min["HA_SW_Low"] == "T") &
    (df_3min["Close"] > df_3min["S1"])  # Confirmed above support

    )
df_3min["strong_sell"] = (
    (df_3min["EMA_5"] < df_3min["EMA_10"]) &
    (df_3min["EMA_10"] < df_3min["EMA_20"]) &
    (df_3min["ADX"] > 25) &
    (df_3min["-DI"] > df_3min["+DI"]) &
    (df_3min["Supertrend"] > df_3min["Close"]) &
    #(df_3min["RSI"] > 40) &
    (df_3min["HA_SW_High"] == "T") &
    (df_3min["Close"] < df_3min["R1"])  # Rejected at resistance
)
df_3min["Final_Signal"] = np.where(
    df_3min["strong_buy"], "STRONG BUY",
    np.where(df_3min["strong_sell"], "STRONG SELL", df_3min["Yest_H/L"])
    )
# Use previous row's S1 and R1
df_3min["S1_prev"] = df_3min["S1"].shift(1)
df_3min["R1_prev"] = df_3min["R1"].shift(1)
df_3min["ATR"] = atr(df_3min, 10)


# Step 1: Calculate Swing High/Low (exclude current candle)
df_3min["Swing_High_5"] = df_3min["High"].shift(1).rolling(window=5).max()
df_3min["Swing_Low_5"] = df_3min["Low"].shift(1).rolling(window=5).min()
# Signal for candel pattern

#df_today=df_3min[["Close","High","Low", "+DI", "-DI","DI_Diff","Variance", "S1", "R1", "S2","R2","HA_Signal", "ADX", "ADX_Incr", "Signal", "EMA_5", "EMA_10", "EMA_20","Golden/Death"]]
#df_3min=df_3min[["Open","Close","High","Low", "Signal", "Score","S1","R1","S2","R2","Signal_S/R","Entry","Target","StopLoss","ADX_Incr","ADX", "+DI", "-DI", "Variance", "EMA_5", "EMA_10"]]
df_3min=df_3min[["Open","Close","High","Low", "Signal","S1","R1","ATR","Swing_Signal","Final_Signal","Swing_High","Swing_Low","ADX_Incr","ADX", "RSI","+DI", "-DI","HA_SW_High","HA_SW_Low","HA_Trend"]]
# filename = f"nifty50BB_{nifty}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
# df_3min.to_csv(filename)
#df_3min = df_3min[(df_3min["EE_Signal"] == "Buy") & (df_3min["HA_SW_Low"] == "T")] # for buy
#df_3min = df_3min[(df_3min["EE_Signal"] == "Buy")] # for buy
#df_3min = df_3min[(df_3min["HA_SW_Low"] == "T")] # for buy
#df_3min = df_3min[(df_3min["HA_SW_High"] == "T")] # for sell
df_3min.tail(10)



[*********************100%***********************]  1 of 1 completed


,Open,Close,High,Low,Signal,S1,R1,ATR,Swing_Signal,Final_Signal,Swing_High,Swing_Low,ADX_Incr,ADX,RSI,+DI,-DI,HA_SW_High,HA_SW_Low,HA_Trend
Datetime,,,,,,,,,,,,,,,,,,,,
2025-05-06 10:39:00+05:30,24378.0,24394.0,24395.0,24378.0,No Signal,24383.0,24400.0,13.1,SwingHigh-Breakout-BUY,Current Close breaks Previous High- BUY,NaN,NaN,False,19.0,44.0,20.97,22.33,F,T,Sideways
2025-05-06 10:42:00+05:30,24395.0,24397.0,24399.0,24386.0,Strong BUY (4.0),24389.0,24402.0,12.5,SwingHigh-Breakout-BUY,Current Close breaks Previous High- BUY,NaN,NaN,False,18.0,46.0,21.54,21.04,F,F,Sideways
2025-05-06 10:45:00+05:30,24398.0,24399.0,24403.0,24396.0,Strong BUY (4.0),24395.0,24402.0,11.9,Swinglow_Breakout-ShortBuy,Current High Breakout PrevHigh -BUY,24403.0,NaN,False,17.0,47.0,22.68,20.37,F,F,Uptrend
2025-05-06 10:48:00+05:30,24398.0,24388.0,24400.0,24388.0,No Signal,24384.0,24396.0,11.8,Swinglow_Breakout-ShortBuy,Current Close lessthan Previous Low -Sell,NaN,NaN,False,16.0,42.0,21.41,22.97,F,F,Sideways
2025-05-06 10:51:00+05:30,24388.0,24392.0,24396.0,24385.0,No Signal,24386.0,24397.0,11.8,Swinglow_Breakout-ShortBuy,Current Low lessthan Previous Low -Sell,NaN,24385.0,False,15.0,44.0,20.28,23.19,F,F,Sideways
2025-05-06 10:54:00+05:30,24392.0,24395.0,24395.0,24385.0,BUY (2.0),24389.0,24399.0,12.0,Swinglow_Breakout-ShortBuy,No Break,NaN,24385.0,False,15.0,46.0,19.29,22.06,F,T,Sideways
2025-05-06 10:57:00+05:30,24395.0,24403.0,24406.0,24392.0,Strong BUY (4.0),24394.0,24408.0,11.9,Swinglow_Breakout-ShortBuy,Current Close breaks Previous High- BUY,24406.0,NaN,False,14.0,50.0,23.36,20.54,F,F,Sideways
2025-05-06 11:00:00+05:30,24404.0,24394.0,24405.0,24393.0,BUY (2.5),24389.0,24401.0,11.6,Swinglow_Breakout-ShortBuy,No Break,NaN,NaN,False,14.0,46.0,21.97,19.32,F,F,Sideways
2025-05-06 11:03:00+05:30,24394.0,24391.0,24397.0,24389.0,BUY (2.5),24387.0,24395.0,11.3,Swinglow_Breakout-ShortBuy,Current Close lessthan Previous Low -Sell,NaN,NaN,False,13.0,44.0,21.07,20.58,F,F,Sideways
